In [1]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import os
import json
import re
from tqdm import tqdm
tqdm.pandas()
from evaluate import load
from sklearn.model_selection import train_test_split

In [2]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

C:\Users\Student\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Initialize the model
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("ai4bharat/indic-bert", num_labels=2)

In [4]:
# Set device (CUDA or CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Load the data
train_df = pd.read_json(r"D:/BP/data/full_data_train.json")
test_df = pd.read_json(r"D:/BP/data/full_data_test.json")


In [6]:
# For hyperparameter search, use a sample of 10% of the data
hp_train_df = train_df.sample(frac=0.1, random_state=42).reset_index(drop=True)
hp_test_df = test_df.sample(frac=0.1, random_state=42).reset_index(drop=True)

In [7]:
# Custom Dataset class for handling the input data
class LegalDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]
        text_data = entry['text']

        # Combine facts and arguments + judge opinion
        combined_text = " ".join(text_data['facts-and-arguments'] + text_data['judge-opinion'])

        # Tokenize
        encoded_sent = self.tokenizer.encode_plus(
            text=combined_text,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )

        input_ids = torch.tensor(encoded_sent['input_ids'])
        attention_mask = torch.tensor(encoded_sent['attention_mask'])
        label = torch.tensor(entry['label'])

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label}


In [8]:
# Create datasets
train_dataset = LegalDataset(train_df.to_dict('records'), tokenizer)
test_dataset = LegalDataset(test_df.to_dict('records'), tokenizer)
hp_train_dataset = LegalDataset(hp_train_df.to_dict('records'), tokenizer)
hp_test_dataset = LegalDataset(hp_test_df.to_dict('records'), tokenizer)


In [9]:
# Load evaluation metrics
accuracy_metric = load("accuracy")
f1_metric = load("f1")

In [10]:

# Metric computation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="micro")
    return {'accuracy': accuracy["accuracy"], 'f1-score': f1["f1"]}


In [11]:
# Hyperparameter search space for Optuna
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.005, 0.05),
        "adam_beta1": trial.suggest_float("adam_beta1", 0.75, 0.95),
        "adam_beta2": trial.suggest_float("adam_beta2", 0.99, 0.9999),
        "adam_epsilon": trial.suggest_float("adam_epsilon", 1e-9, 1e-7, log=True)
    }

In [12]:
# Training arguments
training_args = TrainingArguments(
    output_dir=os.path.join("TFIDF-INDIC", "output"),
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=os.path.join("TFIDF-INDIC", "logs"),
    evaluation_strategy="epoch",
    logging_steps=250,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1-score",
    save_safetensors=False,
)


# Trainer for hyperparameter search
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=hp_train_dataset,
    eval_dataset=hp_test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)


C:\Users\Student\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Perform hyperparameter search
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize", hp_space=my_hp_space)

[I 2024-10-07 09:34:46,020] A new study created in memory with name: no-name-9611d48c-a034-4bc3-9864-b1f5a0aad3c1
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/3732 [00:00<?, ?it/s]

{'loss': 0.6788, 'grad_norm': 0.5041502714157104, 'learning_rate': 3.823437426552948e-05, 'epoch': 0.2}
{'loss': 0.6673, 'grad_norm': 0.20781925320625305, 'learning_rate': 7.646874853105895e-05, 'epoch': 0.4}
{'loss': 0.669, 'grad_norm': 0.20996421575546265, 'learning_rate': 7.055377726473323e-05, 'epoch': 0.6}
{'loss': 0.6697, 'grad_norm': 0.20376186072826385, 'learning_rate': 6.46388059984075e-05, 'epoch': 0.8}


  0%|          | 0/356 [00:00<?, ?it/s]

{'eval_loss': 0.6663722395896912, 'eval_accuracy': 0.6153305203938115, 'eval_f1-score': 0.6153305203938115, 'eval_runtime': 221.3733, 'eval_samples_per_second': 12.847, 'eval_steps_per_second': 1.608, 'epoch': 1.0}
{'loss': 0.6653, 'grad_norm': 0.5340359807014465, 'learning_rate': 5.872383473208178e-05, 'epoch': 1.0}
{'loss': 0.6674, 'grad_norm': 0.20446573197841644, 'learning_rate': 5.2808863465756056e-05, 'epoch': 1.21}
{'loss': 0.6651, 'grad_norm': 0.46533510088920593, 'learning_rate': 4.689389219943034e-05, 'epoch': 1.41}
{'loss': 0.6703, 'grad_norm': 0.16566941142082214, 'learning_rate': 4.0978920933104617e-05, 'epoch': 1.61}
{'loss': 0.6545, 'grad_norm': 0.20004145801067352, 'learning_rate': 3.506394966677889e-05, 'epoch': 1.81}


...

{'eval_loss': 0.6013708710670471, 'eval_accuracy': 0.7060478199718706, 'eval_f1-score': 0.7060478199718706, 'eval_runtime': 211.5565, 'eval_samples_per_second': 13.443, 'eval_steps_per_second': 1.683, 'epoch': 2.0}
{'loss': 0.6195, 'grad_norm': 8.861154556274414, 'learning_rate': 2.7598464757670056e-06, 'epoch': 2.01}
{'loss': 0.5973, 'grad_norm': 10.730295181274414, 'learning_rate': 2.199812694158441e-06, 'epoch': 2.21}
{'loss': 0.5768, 'grad_norm': 27.689355850219727, 'learning_rate': 1.6397789125498767e-06, 'epoch': 2.41}
{'loss': 0.5472, 'grad_norm': 23.123262405395508, 'learning_rate': 1.0797451309413122e-06, 'epoch': 2.61}
{'loss': 0.5419, 'grad_norm': 35.70475769042969, 'learning_rate': 5.197113493327477e-07, 'epoch': 2.81}


  0%|          | 0/356 [00:00<?, ?it/s]

{'eval_loss': 0.5515127182006836, 'eval_accuracy': 0.7433192686357243, 'eval_f1-score': 0.7433192686357243, 'eval_runtime': 217.4566, 'eval_samples_per_second': 13.078, 'eval_steps_per_second': 1.637, 'epoch': 3.0}


[I 2024-10-08 05:38:26,743] Trial 9 finished with value: 1.4866385372714486 and parameters: {'learning_rate': 7.240116728635521e-06, 'weight_decay': 0.022606675299539355, 'adam_beta1': 0.8211102421288813, 'adam_beta2': 0.9969784333856463, 'adam_epsilon': 2.7064505537870976e-09}. Best is trial 9 with value: 1.4866385372714486.


{'train_runtime': 7022.511, 'train_samples_per_second': 4.251, 'train_steps_per_second': 0.531, 'train_loss': 0.6188587115125257, 'epoch': 3.0}


In [14]:
# Print the best hyperparameters
print("Best Hyperparameters:", best_run)

Best Hyperparameters: BestRun(run_id='9', objective=1.4866385372714486, hyperparameters={'learning_rate': 7.240116728635521e-06, 'weight_decay': 0.022606675299539355, 'adam_beta1': 0.8211102421288813, 'adam_beta2': 0.9969784333856463, 'adam_epsilon': 2.7064505537870976e-09}, run_summary=None)


In [15]:
# Garbage collection to free memory after hyperparameter tuning
del trainer
del training_args
import gc
gc.collect()

119

In [16]:
# Training with best hyperparameters
print("Starting final training...")

# Reload training arguments
training_args = TrainingArguments(
    output_dir=os.path.join("TFIDF-INDIC", "output"),
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=os.path.join("TFIDF-INDIC", "logs"),
    evaluation_strategy="epoch",
    logging_steps=250,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1-score",
    save_safetensors=False,
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

Starting final training...


C:\Users\Student\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Load the best hyperparameters and start training
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

In [18]:
# Train the model
trainer.train()

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/12441 [00:00<?, ?it/s]

{'loss': 0.6874, 'grad_norm': 2.6423277854919434, 'learning_rate': 3.6200583643177606e-06, 'epoch': 0.02}
{'loss': 0.6681, 'grad_norm': 10.021020889282227, 'learning_rate': 7.240116728635521e-06, 'epoch': 0.04}
{'loss': 0.6673, 'grad_norm': 2.565160036087036, 'learning_rate': 7.088535690015733e-06, 'epoch': 0.06}
{'loss': 0.6558, 'grad_norm': 6.731729507446289, 'learning_rate': 6.936954651395947e-06, 'epoch': 0.08}
{'loss': 0.6687, 'grad_norm': 15.670312881469727, 'learning_rate': 6.785373612776159e-06, 'epoch': 0.1}
{'loss': 0.6509, 'grad_norm': 3.0118961334228516, 'learning_rate': 6.633792574156372e-06, 'epoch': 0.12}
{'loss': 0.6631, 'grad_norm': 2.480739116668701, 'learning_rate': 6.482211535536585e-06, 'epoch': 0.14}
{'loss': 0.6418, 'grad_norm': 8.68790340423584, 'learning_rate': 6.330630496916797e-06, 'epoch': 0.16}
{'loss': 0.6095, 'grad_norm': 20.069482803344727, 'learning_rate': 6.1790494582970105e-06, 'epoch': 0.18}
{'loss': 0.5676, 'grad_norm': 14.53165054321289, 'learning_

  0%|          | 0/3555 [00:00<?, ?it/s]

{'eval_loss': 0.43305346369743347, 'eval_accuracy': 0.7984596989731326, 'eval_f1-score': 0.7984596989731326, 'eval_runtime': 2219.5695, 'eval_samples_per_second': 12.811, 'eval_steps_per_second': 1.602, 'epoch': 1.0}
{'train_runtime': 23687.7936, 'train_samples_per_second': 4.201, 'train_steps_per_second': 0.525, 'train_loss': 0.5067445521818059, 'epoch': 1.0}


TrainOutput(global_step=12441, training_loss=0.5067445521818059, metrics={'train_runtime': 23687.7936, 'train_samples_per_second': 4.201, 'train_steps_per_second': 0.525, 'total_flos': 2378386883358720.0, 'train_loss': 0.5067445521818059, 'epoch': 1.0})

In [19]:
trainer.save_model(r"D:/BP/model")